# Lab 6 - Deploy a model as REST endpoint

Azure ML provides some useful functionality to speed up operationalization of ML models; a task that organizations have struggled with in the past. 

In this lab, we are going to deploy our model to an Azure Container Instance (ACI), a service that enables a developer to deploy containers on the Microsoft Azure public cloud without having to provision or manage any underlying infrastructure. This is suitable for test & dev deployment scenarios; for production deployment one of the other [deployment target options](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target#deploy) should be considered.

After connecting to our Azure ML workspace yet again, we are downloading our previously registered model which will then be encapsulated into the docker image that's going to be run in ACI.

In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model
import os

ws = Workspace.from_config()
model = Model(ws, 'german-credit-local-model')
model.download(target_dir=os.getcwd(), exist_ok=True)

# verify the downloaded model file
file_path = os.path.join(os.getcwd(), "model.pkl")

os.stat(file_path)

## Specifiying dependencies

Dependencies are an important aspect of operationalizing ML models. We must ensure that all libraries that were required for model training are also made available as a part of the inference environment which we are about to provision. Azure ML facilitates dealing with these [environments](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments) and provides some out-of-the-box environments as well. Here, we will be creating our own custom environment which we write to a file called *myenv.yml*. If you look at the location of where this Jupyter Notebook sits, you will see the file stored there as well once the code has run.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

with open("myenv.yml", "w") as f:
    f.write(myenv.serialize_to_string())

Let's quickly look at what dependencies are made part of our environment.

In [ ]:
with open("myenv.yml", "r") as f:
    print(f.read())

## Webservice configuration

As mentioned in the beginning, we are going to create an Azure Container Instance based webservice to run our model. The following code cell defines the configuration for the ACI, including the number of cores and amount of memory that should be allocated.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "german-credit-data",  
                                                     "framework": "sklearn"},
                                               description='Predict credit risk with sklearn')

## Deploy endpoint

Finally, we are going to deploy the endpoint for our model. This will embed our custom environment into a Docker image, register this Docker image in the Azure Container Registry of our Azure ML workspace and then spin up an ACI instance for us. All of these steps combined will take about 10 minutes to complete. 

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="score.py",
                                   conda_file="myenv.yml")

service = Model.deploy(workspace=ws, 
                       name='german-credit-risk',
                       models=[model], 
                       inference_config=inference_config,
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

We can quickly check the service's health as well as the URI that it is now available under. Note that you can also retrieve this URI from [Azure ML Studio](https://ml.azure.com) when navigating to the *endpoints* page.

This URI may now be used to score new credit requests and get the ML models prediction on whether the credit will be at risk or not.

In [ ]:
print(service.state)
print(service.scoring_uri)

## Conclusion

In this lab you have learned how to deploy a previously trained ML model for operationalization. This is the final and very important step since previously the model was only available inside of python. In order to really gain business value from it, however, it must be made available to the outside world. This is accomplished by deploying the model as a web service inside of a Docker image, enabling LOB applications such as ERPs to interact with the model and incorporate its prediction into a business process. 

Please note that we have used [Azure Container Instances (ACI)](https://docs.microsoft.com/azure/container-instances/) to host our Docker images which is suitable only for dev/test purposes. If you intend to operationalize your model in a productive manner, it is strongly advised to use [Azure Kubernetes Services (AKS)](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-azure-kubernetes-service) instead. 

## Disclaimer

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.